In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
from textblob import TextBlob
import spacy
from tqdm import tqdm
import pickle
import re
from sklearn import metrics
import scipy as sp
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV


## Importing the file with the raw text data for the purpose of building a Pipeline and finding the best parameters with selected Classifier

In [0]:
df= pickle.load(open("save_modifieddatabase_trainData.pkl","rb"))

In [0]:
df_X = df['reviewContent']
df_y = pickle.load(open("save_preprocess_trainData_labels.pkl","rb"))

In [0]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(df_X, df_y)

## Description of Parameters in the code below 

vect__ngram_range: [(1, 1), (1, 2)] - To Decide whether to go with Unigram or Bigram approach. For the record tried Trigram also but it didnot yieled good results, so inorder to make the code run faster for the final selction it has been removed from the hyperparameters mentioned below.
n_jobs=-1:This is used for parallel processing in the case of heavy computation, so as to decrease the total execution time

In [0]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1) 
gs_clf = gs_clf.fit(df_X, df_y)

/Users/neha/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
print("Grid Search Best Score: ",gs_clf.best_score_)
print("Grid Search Best Parameters: ",gs_clf.best_params_)

Grid Search Best Score:  0.679635279596924
Grid Search Best Parameters:  {'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


In [0]:
from sklearn.externals import joblib
joblib.dump(gs_clf, 'Naive_Bayes_GS.skmodel')

/Users/neha/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['Naive_Bayes_GS.skmodel']